In [3]:
import pandas as pd

df = pd.read_pickle("training_data/grammar_data_df.pkl")
df.head()

,parser_state,syncode_logprobs,baseline_logprobs
0,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -in...","[-10.92, -4.24, -10.01, -3.38, -6.41, -8.39, -..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -in...","[-10.92, -4.24, -10.01, -3.38, -6.41, -8.39, -..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -in...","[-10.92, -4.24, -10.01, -3.38, -6.41, -8.39, -..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -in...","[-10.92, -4.24, -10.01, -3.38, -6.41, -8.39, -..."
4,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -in...","[-10.92, -4.24, -10.01, -3.38, -6.41, -8.39, -..."


In [5]:
import numpy as np
arr = np.stack(df['syncode_logprobs'].values)

finite_mask = np.isfinite(arr)
rows_with_non_inf = finite_mask.any(axis=1)

print("Rows with ≥1 non -inf value:", np.where(rows_with_non_inf)[0])
print(f"{rows_with_non_inf.sum()}/{arr.shape[0]} rows have at least one non -inf element")

Rows with ≥1 non -inf value: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84]
85/85 rows have at least one non -inf element


In [7]:
def extract_noninf(arr):
    """
    Returns (first_noninf_value, its_index) or (np.nan, None) if none found.
    """
    for idx, val in enumerate(arr):
        if np.isfinite(val):
            return val, idx
    return np.nan, None

In [8]:
df[['syncode_prob','syncode_idx']] = (
    df['syncode_logprobs']
      .apply(lambda v: pd.Series(extract_noninf(v), index=['syncode_prob','syncode_idx']))
)

new_df = df[['parser_state','syncode_prob','syncode_idx','baseline_logprobs']]

# Inspect:
new_df.head()

,parser_state,syncode_prob,syncode_idx,baseline_logprobs
0,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]",-5.367188,58.0,"[-10.92, -4.24, -10.01, -3.38, -6.41, -8.39, -..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",-0.008553,151643.0,"[-10.92, -4.24, -10.01, -3.38, -6.41, -8.39, -..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",-0.008553,151643.0,"[-10.92, -4.24, -10.01, -3.38, -6.41, -8.39, -..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]",-0.008553,151643.0,"[-10.92, -4.24, -10.01, -3.38, -6.41, -8.39, -..."
4,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",-0.008553,151643.0,"[-10.92, -4.24, -10.01, -3.38, -6.41, -8.39, -..."
